In [1]:
import Pkg
Pkg.add("POMDPs")
Pkg.add("POMDPPolicies")
Pkg.add("POMDPModelTools")
Pkg.add("POMDPSimulators")

  Updating registry at `C:\Users\rmoss\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating registry at `C:\Users\rmoss\.julia\registries\JuliaPOMDP`
  Updating git-repo `https://github.com/JuliaPOMDP/Registry`
 Resolving package versions...
  Updating `C:\Users\rmoss\Code\sisl\aa228\aa228-notebook\Project.toml`
 [no changes]
  Updating `C:\Users\rmoss\Code\sisl\aa228\aa228-notebook\Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `C:\Users\rmoss\Code\sisl\aa228\aa228-notebook\Project.toml`
 [no changes]
  Updating `C:\Users\rmoss\Code\sisl\aa228\aa228-notebook\Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `C:\Users\rmoss\Code\sisl\aa228\aa228-notebook\Project.toml`
 [no changes]
  Updating `C:\Users\rmoss\Code\sisl\aa228\aa228-notebook\Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `C:\Users\rmoss\Code\sisl\aa228\aa228-notebook\Project.toml`
 [no changes]
  Upda

In [2]:
using POMDPs
using POMDPPolicies
using POMDPModelTools
using POMDPSimulators

In [3]:
include("helpers.jl");

# Chain MDP

State space: Integer representing which node we are in.

Action space: Try to move left or right (represented by symbols `:left` and `:right`.

Transitions: Move in action direction with probability `p_success`, otherwise move in the opposite direction.

Reward: +1 for being in rightmost state

In [4]:
plot_chain(4, fill=Dict(4=>"green"))

First, create a new type that represents the MDP

In [5]:
struct ChainMDP <: MDP{Int, Symbol}
    len::Int
    p_success::Float64
    discount::Float64
end

We define the behavior of the MDP by imlementing functions from the POMDPs.jl API (http://juliapomdp.github.io/POMDPs.jl/latest/api/). 

In [6]:
?SparseCat

search: SparseCat



```
SparseCat(values, probabilities)
```

Create a sparse categorical distribution.

`values` is an iterable object containing the possible values (can be of any type) in the distribution that have nonzero probability. `probabilities` is an iterable object that contains the associated probabilities.

This is optimized for value iteration with a fast implementation of `weighted_iterator`. Both `pdf` and `rand` are order n.


In [7]:
# the transition function returns the distribution of sp given that
# action a is taken in state s
function POMDPs.transition(mdp::ChainMDP, s::Int, a::Symbol)
    if a == :right
        success = min(s+1, mdp.len)
        failure = max(s-1, 1)
    else # a == :left
        success = max(s-1, 1)
        failure = min(s+1, mdp.len)
    end
    return SparseCat([success, failure], [mdp.p_success, 1.0-mdp.p_success])
end

Note: this allocates arrays in the return statement - it could be sped up by using [StaticArrays.jl](https://github.com/JuliaArrays/StaticArrays.jl)

In [8]:
function POMDPs.reward(mdp::ChainMDP, s::Int, a::Symbol)
    if s == mdp.len
        return 1.0
    else 
        return 0.0
    end
end

In [9]:
POMDPs.discount(mdp::ChainMDP) = mdp.discount

In [10]:
POMDPs.stateindex(::ChainMDP, s::Int64) = s

And that's all we need to define the MDP (for simulation at least)

In [11]:
mdp = ChainMDP(4, 0.7, 0.9)

ChainMDP(4, 0.7, 0.9)

In [12]:
transition(mdp, 1, :right)

              SparseCat distribution
     ┌                                        ┐ 
   2 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.7   
   1 ┤■■■■■■■■■■■■■■■ 0.30000000000000004       
     └                                        ┘ 

In [13]:
reward(mdp, 1, :right)

0.0

In [14]:
reward(mdp, 4, :left)

1.0

# Simulations

There are three basic ways to simulate a POMDP.



If we want to try out simple policies we can construct a `Policy` type and implement the `POMDPs.action` function which defines the policy.

In [15]:
mutable struct SimplePolicy <: Policy end
POMDPs.action(::SimplePolicy, s::Int64) = (s == 2 ? :left : :right)
p = SimplePolicy()

SimplePolicy()

In [16]:
hr = HistoryRecorder(max_steps=10, show_progress=true)
history = simulate(hr, mdp, p, 1);

Simulating...100%|██████████████████████████████████████| Time: 0:00:01


In [17]:
for (s, a, r) in eachstep(history, "s,a,r")
    @printf("State: %2d    Action: %6s   Reward: %1d\n", s, a, r)
end    

State:  1    Action:  right   Reward: 0
State:  1    Action:  right   Reward: 0
State:  2    Action:   left   Reward: 0
State:  1    Action:  right   Reward: 0
State:  2    Action:   left   Reward: 0
State:  1    Action:  right   Reward: 0
State:  1    Action:  right   Reward: 0
State:  1    Action:  right   Reward: 0
State:  2    Action:   left   Reward: 0
State:  1    Action:  right   Reward: 0


We can also specify a policy object, for example using a vector

In [18]:
@show policy_vector = fill(:right, 4);

policy_vector = fill(:right, 4) = Symbol[:right, :right, :right, :right]


In [19]:
policy = VectorPolicy(mdp, policy_vector);

and step through a simulation as it is being performed

In [20]:
for (s, a, r) in stepthrough(mdp, policy, 1, "s,a,r", max_steps=10)
    @printf("State: %2d    Action: %6s   Reward: %1d\n", s, a, r)
    sleep(1.0)
end

State:  1    Action:  right   Reward: 0
State:  2    Action:  right   Reward: 0
State:  1    Action:  right   Reward: 0
State:  2    Action:  right   Reward: 0
State:  3    Action:  right   Reward: 0
State:  4    Action:  right   Reward: 1
State:  3    Action:  right   Reward: 0
State:  4    Action:  right   Reward: 1
State:  4    Action:  right   Reward: 1
State:  4    Action:  right   Reward: 1


And also use a simulator object and the `simulate()` function.

In [21]:
simulator = HistoryRecorder(max_steps=10, show_progress=true)
history = simulate(simulator, mdp, policy, 1)
@show state_hist(history);

state_hist(history) = [1, 2, 3, 2, 3, 4, 3, 4, 3, 4, 4]


# Solving with MCTS


The required API functions depend on the solver. Suppose we want to use the MCTS solver from MCTS.jl:

In [22]:
#Pkg.add("MCTS")
using MCTS

In [23]:
solver = MCTSSolver()
mdp = ChainMDP(4, 0.8, 0.95)

@requirements_info solver mdp

INFO: POMDPs.jl requirements for action(::AbstractMCTSPlanner, ::Any) and dependencies. ([✔] = implemented correctly; [X] = missing)

For action(::AbstractMCTSPlanner, ::Any):
  [No additional requirements]
For simulate(::AbstractMCTSPlanner, ::Any, ::Int64) (in action(::AbstractMCTSPlanner, ::Any)):
  [✔] discount(::ChainMDP)
  [✔] isterminal(::ChainMDP, ::Int64)
  [✔] generate_sr(::ChainMDP, ::Int64, ::Symbol, ::MersenneTwister)
  [✔] isequal(::Int64, ::Int64)
  [✔] hash(::Int64)
For insert_node!(::MCTSPlanner, ::Int64) (in simulate(::AbstractMCTSPlanner, ::Any, ::Int64)):
  [No requirements specified]
For estimate_value(::MCTS.SolvedRolloutEstimator, ::MDP, ::Any, ::Int64) (in simulate(::AbstractMCTSPlanner, ::Any, ::Int64)):
  [No additional requirements]
For rollout(::MCTS.SolvedRolloutEstimator, ::MDP, ::Any, ::Int64) (in estimate_value(::MCTS.SolvedRolloutEstimator, ::MDP, ::Any, ::Int64)):
  [No additional requirements]
For simulate(::RolloutSimulator, ::MDP, ::Policy, ::Any) (

true

In [24]:
?actions

search: actions n_actions action_hist actionvalues ordered_actions



```
actions(problem::POMDP)
actions(problem::MDP)
```

Returns the entire action space of a POMDP.

---

```
actions{S,A,O}(problem::POMDP{S,A,O}, state::S)
actions{S,A}(problem::MDP{S,A}, state::S)
```

Return the action space accessible from the given state.

---

```
actions(problem::POMDP, belief)
```

Return the action space accessible from the states with nonzero belief.


In [25]:
POMDPs.actions(::ChainMDP) = [:left, :right]

In [26]:
@requirements_info solver mdp

INFO: POMDPs.jl requirements for action(::AbstractMCTSPlanner, ::Any) and dependencies. ([✔] = implemented correctly; [X] = missing)

For action(::AbstractMCTSPlanner, ::Any):
  [No additional requirements]
For simulate(::AbstractMCTSPlanner, ::Any, ::Int64) (in action(::AbstractMCTSPlanner, ::Any)):
  [✔] discount(::ChainMDP)
  [✔] isterminal(::ChainMDP, ::Int64)
  [✔] generate_sr(::ChainMDP, ::Int64, ::Symbol, ::MersenneTwister)
  [✔] isequal(::Int64, ::Int64)
  [✔] hash(::Int64)
For insert_node!(::MCTSPlanner, ::Int64) (in simulate(::AbstractMCTSPlanner, ::Any, ::Int64)):
  [No requirements specified]
For estimate_value(::MCTS.SolvedRolloutEstimator, ::MDP, ::Any, ::Int64) (in simulate(::AbstractMCTSPlanner, ::Any, ::Int64)):
  [No additional requirements]
For rollout(::MCTS.SolvedRolloutEstimator, ::MDP, ::Any, ::Int64) (in estimate_value(::MCTS.SolvedRolloutEstimator, ::MDP, ::Any, ::Int64)):
  [No additional requirements]
For simulate(::RolloutSimulator, ::MDP, ::Policy, ::Any) (

true

In [27]:
solver = MCTSSolver(enable_tree_vis=true)
planner = solve(solver, mdp);

In [28]:
simulate(simulator, mdp, planner, 1)

MDPHistory{Int64,Symbol}([1, 2, 3, 4, 4, 3, 4, 4, 3, 2, 3], Symbol[:right, :right, :right, :right, :right, :right, :right, :right, :right, :right], [0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0], Any[nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing], Any[nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing], 0.95, nothing, nothing)

In [29]:
using D3Trees
D3Tree(planner, 1)

1
N:     53

├──                     left
│  Q:   0.00
│  N:      7

│  ├──                        2
│  │  N:    179

│  │  ├──                     left
│  │  │  Q:   0.32
│  │  │  N:      4
 (1 children)
│  │  └──                    right
│  │     Q:   3.01
│  │     N:    175
 (2 children)
│  └──                        1
│     N:     53

│     ├──                     left
│     │  Q:   0.00
│     │  N:      7
 (2 children)
│     └──                    right
│        Q:   1.98
│        N:     46
 (2 children)
└──                    right
   Q:   1.98
   N:     46

   ├──                        2
   │  N:    179

   │  ├──                     left
   │  │  Q:   0.32
   │  │  N:      4
 (1 children)
   │  └──                    right
   │     Q:   3.01
   │     N:    175
 (2 children)
   └──                        1
      N:     53

      ├──                     left
      │  Q:   0.00
      │  N:      7
 (2 children)
      └──                    right
         Q:   1.98
         N:     46
 (2 children)

# Solving with value iteration

We can also try value iteration from the DiscreteValueIteration package

In [30]:
using DiscreteValueIteration

In [31]:
using DiscreteValueIteration
solver = ValueIterationSolver(verbose=true)
try
    @requirements_info solver mdp
catch err
    @warn err
end

INFO: POMDPs.jl requirements for solve(::ValueIterationSolver, ::Union{MDP,POMDP}) and dependencies. ([✔] = implemented correctly; [X] = missing)

For solve(::ValueIterationSolver, ::Union{MDP,POMDP}):
  [✔] discount(::ChainMDP)
  [X] n_states(::ChainMDP)
  [X] n_actions(::ChainMDP)
  [✔] transition(::ChainMDP, ::Int64, ::Symbol)
  [✔] reward(::ChainMDP, ::Int64, ::Symbol, ::Int64)
  [✔] stateindex(::ChainMDP, ::Int64)
  [X] actionindex(::ChainMDP, ::Symbol)
  [✔] actions(::ChainMDP, ::Int64)
For ordered_states(::Union{MDP,POMDP}) (in solve(::ValueIterationSolver, ::Union{MDP,POMDP})):
  [X] states(::ChainMDP)
For ordered_actions(::Union{MDP,POMDP}) (in solve(::ValueIterationSolver, ::Union{MDP,POMDP})):
  [✔] actions(::ChainMDP)
Note: Missing methods are often due to incorrect importing. You must explicitly import POMDPs functions to add new methods.

Throwing the first exception (from processing solve(::ValueIterationSolver, ::Union{MDP,POMDP}) requirements):



┌ Warning: ErrorException("rethrow(exc) not allowed outside a catch block")
└ @ Main In[31]:6


In [32]:
POMDPs.n_states(mdp::ChainMDP) = mdp.len
POMDPs.states(mdp::ChainMDP) = 1:mdp.len
POMDPs.n_actions(::ChainMDP) = 2
POMDPs.actionindex(::ChainMDP, a::Symbol) = a == :left ? 1 : 2

In [33]:
@requirements_info solver mdp

INFO: POMDPs.jl requirements for solve(::ValueIterationSolver, ::Union{MDP,POMDP}) and dependencies. ([✔] = implemented correctly; [X] = missing)

For solve(::ValueIterationSolver, ::Union{MDP,POMDP}):
  [✔] discount(::ChainMDP)
  [✔] n_states(::ChainMDP)
  [✔] n_actions(::ChainMDP)
  [✔] transition(::ChainMDP, ::Int64, ::Symbol)
  [✔] reward(::ChainMDP, ::Int64, ::Symbol, ::Int64)
  [✔] stateindex(::ChainMDP, ::Int64)
  [✔] actionindex(::ChainMDP, ::Symbol)
  [✔] actions(::ChainMDP, ::Int64)
  [✔] support(::SparseCat)
  [✔] pdf(::SparseCat, ::Int64)
For ordered_states(::Union{MDP,POMDP}) (in solve(::ValueIterationSolver, ::Union{MDP,POMDP})):
  [✔] states(::ChainMDP)
For ordered_actions(::Union{MDP,POMDP}) (in solve(::ValueIterationSolver, ::Union{MDP,POMDP})):
  [✔] actions(::ChainMDP)


true

In [34]:
vi_policy = solve(solver, mdp)

[Iteration 1   ] residual:          1 | iteration runtime:      0.005 ms, (  5.3E-06 s total)
[Iteration 2   ] residual:      0.904 | iteration runtime:      0.005 ms, (  1.06E-05 s total)
[Iteration 3   ] residual:      0.839 | iteration runtime:      0.002 ms, (  1.26E-05 s total)
[Iteration 4   ] residual:      0.783 | iteration runtime:      0.004 ms, (  1.66E-05 s total)
[Iteration 5   ] residual:      0.734 | iteration runtime:      0.002 ms, (  1.9E-05 s total)
[Iteration 6   ] residual:      0.688 | iteration runtime:      0.002 ms, (  2.1E-05 s total)
[Iteration 7   ] residual:      0.646 | iteration runtime:      0.002 ms, (  2.3E-05 s total)
[Iteration 8   ] residual:      0.609 | iteration runtime:      0.002 ms, (  2.52E-05 s total)
[Iteration 9   ] residual:      0.576 | iteration runtime:      0.003 ms, (  2.78E-05 s total)
[Iteration 10  ] residual:      0.543 | iteration runtime:      0.003 ms, (  3.1E-05 s total)
[Iteration 11  ] residual:       0.51 | iteration runti

ValueIterationPolicy{Float64}:
 1 -> :right
 2 -> :right
 3 -> :right
 4 -> :right

In [35]:
action(vip::ValueIterationPolicy, s::Int64) = vip.policy[s]

action (generic function with 1 method)

In [36]:
for s in states(mdp)
    println("State: $s Action: $(action(vi_policy, s))")
end

State: 1 Action: 2
State: 2 Action: 2
State: 3 Action: 2
State: 4 Action: 2


# Crying Baby POMDP

In [37]:
using POMDPModels

In [38]:
bp = BabyPOMDP()
up = updater(bp)

BeliefUpdaters.DiscreteUpdater{BabyPOMDP}(BabyPOMDP(-5.0, -10.0, 0.1, 0.8, 0.1, 0.9))

In [39]:
Pkg.add("SARSOP")
using SARSOP # might take a while to compile

 Resolving package versions...
  Updating `C:\Users\rmoss\Code\sisl\aa228\aa228-notebook\Project.toml`
 [no changes]
  Updating `C:\Users\rmoss\Code\sisl\aa228\aa228-notebook\Manifest.toml`
 [no changes]


In [40]:
solver = SARSOPSolver()

SARSOPSolver
  fast: Bool false
  randomization: Bool false
  precision: Float64 0.001
  timeout: Nothing nothing
  memory: Nothing nothing
  trial_improvement_factor: Float64 0.5
  policy_interval: Nothing nothing
  verbose: Bool true
  policy_filename: String "policy.out"
  pomdp_filename: String "model.pomdpx"


In [41]:
POMDPPolicies.AlphaVectorPolicy(p::POMDP, alphas::Matrix{Float64}, action_map::BitVector) = POMDPPolicies.AlphaVectorPolicy(p, alphas, Array(action_map))
p = solve(solver, bp)

Generating a pomdpx file: model.pomdpx

Loading the model ...
  input file   : model.pomdpx
  loading time : 0.00s 

SARSOP initializing ...
  initialization time : 0.00s

-------------------------------------------------------------------------------
 Time   |#Trial |#Backup |LBound    |UBound    |Precision  |#Alphas |#Beliefs  
-------------------------------------------------------------------------------
 0.001   0       0        -47.3685   -16.0713   31.2971     2        1        
 0.002   5       51       -17.9323   -16.0492   1.88318     9        6        
 0.003   10      103      -16.3978   -16.2159   0.181953    2        8        
 0.004   14      151      -16.3124   -16.277    0.0353847   2        13       
 0.005   18      207      -16.3058   -16.2969   0.00887227  2        16       
 0.005   21      250      -16.3055   -16.3022   0.00329006  7        16       
 0.007   25      305      -16.3055   -16.3044   0.00111584  2        18       
 0.007   26      323      -16.3055 

AlphaVectorPolicy{BabyPOMDP,Bool}(BabyPOMDP(-5.0, -10.0, 0.1, 0.8, 0.1, 0.9), Array{Float64,1}[[-19.6749, -29.6749], [-16.3055, -38.2512]], Bool[1, 0])

In [42]:
history = simulate(simulator, bp, p, up, BoolDistribution(0.0));

Simulating...100%|██████████████████████████████████████| Time: 0:00:00


In [43]:
for (b, a) in eachstep(history, "b,a")
    @printf("%3.2f hungry belief => %s\n",
            b.b[1],
            a ? "feed" : "don't feed")
end

1.00 hungry belief => don't feed
0.98 hungry belief => don't feed
0.97 hungry belief => don't feed
0.97 hungry belief => don't feed
0.97 hungry belief => don't feed
0.97 hungry belief => don't feed
0.46 hungry belief => feed
1.00 hungry belief => don't feed
0.98 hungry belief => don't feed
0.47 hungry belief => feed
